## **0. Introduction**

This notebook continues the series of machine learning techniques with the goal to decode the brain activitiy given the respective categories. Unfortunately, neither the LogReg nor the SVM produced any results that back the initial hypothesis. Still, we will set up the Neuralnetwork for the purpose of learning the respective technique.

## **1.0 Loading the data**

In [1]:
import os
os.chdir("/home/jpauli/ds001506/sourcedata/sub-01/anat")

In [2]:
mask_img_path = '/home/jpauli/ds001506/sourcedata/sub-01/anat'
mask_img_L = os.path.join(mask_img_path,'sub-01_mask_LH_V1v.nii.gz')
mask_img_R = os.path.join(mask_img_path,'sub-01_mask_RH_LOC.nii.gz')

In [3]:
from nilearn.maskers import NiftiMasker
nifti_masker = NiftiMasker(mask_img=mask_img_L)

In [432]:
import numpy as np
import pandas as pd
X = []
Y = []
Sessions = []
category = 1
for session in ["1","2","3","4"]:
    os.chdir('/mnt/c/Users/janos/git/sessions_new/z_maps_{}'.format(session))
    for x in os.listdir():
        if x == 'nilearn_cache':
            continue
        else:
            X.append(nifti_masker.fit_transform(x))
            Y.append(category)
            Sessions.append(session)
            category=category+1
            if category == 27:
                category = 1

Y = np.array(Y)
df =  pd.DataFrame(np.concatenate(X))
X_ = df.to_numpy()

KeyboardInterrupt: 

Split data

In [403]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X_, Y, test_size=0.2,random_state=42)

General imports of pytorch modules needed to define the network. NN stands for Neural Network and is part of the pytorch library.

In [5]:
import torch
import torch.nn.functional as F
from torch import nn

Numpy arrays to pytorch tensor

In [431]:
X_tensor_train = torch.from_numpy(X_train)
Y_tensor_train = torch.from_numpy(y_train)
X_tensor = torch.from_numpy(X_)
Y_tensor = torch.from_numpy(Y)

TypeError: expected np.ndarray (got DataFrame)

In [419]:
Y_tensor_train = Y_tensor_train.type(torch.LongTensor)
X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)

XX = torch.tensor(X_tensor,dtype = torch.float32)
YYY = Y_tensor.type(torch.LongTensor)

/home/jpauli/miniconda3/envs/neuro_ai/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/jpauli/miniconda3/envs/neuro_ai/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


First things first, we should define our neural network by using pythons *classes* and more important, the pytorch nn.Module. Then, we initialize and determine the inputs: The class takes three arguments: Itself (so nn.Module), n inputs and n outputs, corresponding to our x and y data. 
We apply the super function in order to invoke the properties of the class.

Then we define our first variables: "self.in_layer" and "self.out_layer".
By using the linear function of the nn module, we define for the first variable, that it takes n inputs and n hidden layers. This is our input layer, so the information is forwarded from the n_input to n_hidden layers. The out_layer variable passed the input from the n_hidden layers to 26 output layers. 26 is chosen, because we want to predict 26 classes.

We go on by defining the forward method. The forward method runs when calling the class as a function. It takes the z_maps as an input and passes it through the hidden layer to produce the weights and to finally pass it to the output layer, in order to predict the respective category.

In [338]:
class DecodingNN(nn.Module):
    def __init__(self,n_inputs,n_hidden):
        super().__init__()
        self.in_layer = nn.linear(n_inputs,n_hidden)
        self.out_layer = nn.linear(n_hidden, 26)
    def forward(self,V_P):
        h = self.in_layer(V_P)
        y = self.out_layer(h)
        return y


In [428]:
class DecodingReLuNN(nn.Module):
    def __init__(self,n_inputs,n_hidden):
        super().__init__()
        self.in_layer = nn.Linear(n_inputs,n_hidden)
        self.out_layer = nn.Linear(n_hidden, 4)
    def forward(self,V_P):
    #Decode category from voxel pattern

    #Args:
     # Voxel_pattern (torch.Tensor): vector of neural responses to decode, must be of
      #  length n_inputs. Can also be a tensor of shape n_stimuli x n_inputs,
       # containing n_stimuli vectors of neural responses

    #Returns:
    #torch.Tensor: network outputs for each input provided in Voxel_pattern. If
    #Voxel_pattern is a vector, then y is a 1D tensor of length 1. If Voxel_pattern is a 2D
    #tensor then y is a 2D tensor of shape n_stimuli x 1.
    
        h = torch.relu(self.in_layer(V_P))
        y = self.out_layer(h)
        return y

In [430]:
np.random.seed(1)
torch.manual_seed(1)

#Initialize a deep network with 10 hidden units
net = DecodingReLuNN(X_tensor.shape[1], 10)

# Get voxel pattern and predicted category to one stimulus in dataset
V_P, category = XX[[0,26,52,78]], Y_tensor[[0,26,52,78]]  # using helper function get_data

# Decode category from these neural responses using initialized network
out = net(V_P)  # compute output from network, equivalent to net.forward(r)
loss_fn = nn.CrossEntropyLoss()

# Evaluate mean squared error
loss = loss_fn(out, category)

print('decoded category: {}'.format(out))
print('true category: {}'.format(category))
print('cross entropy loss {}'.format(loss))

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [425]:
YYY[[0,26,52,78]]

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])

In [382]:
print('decoded category: {}'.format(out.shape))
print('true category: {}'.format(category.shape))
print('cross entropy loss {}'.format(loss.shape))

decoded category: torch.Size([26, 26])
true category: torch.Size([26])
cross entropy loss torch.Size([])
